# KH-5 ARGON GCPs retrieval and storage
The objective of this script is to retrieve all the GCPs defined for each available ARGON image, and merge them into a single `.shp` file, in order to display them with QGIS.

In [1]:
import os
import posixpath
import posixpath
import pandas as pd
import geopandas as gpd

List of available KH-5 ARGON images:

In [2]:
images_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(images_root):
    if os.path.isdir(posixpath.join(images_root, x)):
        products.append(x)
        
products

['DS09034A007MC018',
 'DS09034A007MC019',
 'DS09034A008MC019',
 'DS09034A008MC020',
 'DS09034A008MC021',
 'DS09034A008MC022',
 'DS09058A024MC012',
 'DS09058A024MC013']

Find and concatenate all the GCPs:

In [3]:
all_GCPs = []
for product in products:
    file = posixpath.join(images_root, product, "GCP_4326.points")
    if os.path.exists(file):
        GCPs_p = pd.read_csv(file, encoding="windows-1252", skiprows=1)
        GCPs_p.columns = ["lon", "lat", "x_img", "y_img", "enable", "dX", "dY", "residual"]
        GCPs_p.drop(columns=["enable", "dX", "dY", "residual"], inplace=True)
        GCPs_p.loc[:, "y_img"] = - GCPs_p.loc[:, "y_img"]
        GCPs_p.loc[:, "image"] = product
        all_GCPs.append(GCPs_p)

GCPs = pd.concat(all_GCPs, ignore_index=True)
GCPs.describe(percentiles=[])

,lon,lat,x_img,y_img
count,1403.000000,1403.000000,1403.000000,1403.000000
mean,16.090324,78.803230,10773.689829,9865.912033
std,2.716894,0.723730,3164.442693,4197.171501
min,11.092010,76.443669,2201.319577,922.504687
50%,15.947072,78.869887,10252.886173,10372.436077
max,25.785755,80.501482,17590.888720,17161.479167


Save as a `.shp` file:

In [4]:
GCPs_gpd = gpd.GeoDataFrame(
    GCPs,
    geometry=gpd.points_from_xy(GCPs.lon, GCPs.lat),
    crs="EPSG:4326"
)

GCPs_gpd.to_file(
    posixpath.join(images_root, "GCPs.shp"),
    driver="ESRI Shapefile"
)

In [5]:
GCPs.loc[GCPs.image == 'DS09034A007MC018']

,lon,lat,x_img,y_img,image
0,16.828623,78.613437,17180.953284,10874.721169,DS09034A007MC018
1,16.690747,78.586144,17272.886763,10920.020451,DS09034A007MC018
2,16.869854,78.567359,17293.056684,10806.875159,DS09034A007MC018
3,16.505390,78.511680,17496.380464,10949.092358,DS09034A007MC018
4,16.746596,78.535399,17401.090741,10851.662437,DS09034A007MC018
...,...,...,...,...,...
173,11.511732,79.207314,16371.920245,14142.307307,DS09034A007MC018
174,11.092010,79.362530,15996.179126,14463.403699,DS09034A007MC018
175,20.205513,78.498023,16820.127270,8967.180736,DS09034A007MC018
176,20.830168,78.325934,17123.885386,8443.117879,DS09034A007MC018


In [6]:
for product in products:
    print(f"{product}: {len(GCPs.loc[GCPs.image == product])} GCPs")

DS09034A007MC018: 178 GCPs
DS09034A007MC019: 134 GCPs
DS09034A008MC019: 0 GCPs
DS09034A008MC020: 267 GCPs
DS09034A008MC021: 241 GCPs
DS09034A008MC022: 252 GCPs
DS09058A024MC012: 167 GCPs
DS09058A024MC013: 164 GCPs
